This script reads a shapefile of US Counties provided by the NWS (https://www.weather.gov/gis/Counties), extracts the longitude and latitude of the centroid of each county, rounds to the nearest 0.25 degree (to be compatible with the ERA5 data set) and also includes the predictors:
- State
- CWA (County Warning Area)
- County Name
- FIPS (the official county code)
- FE_Area (the area of the state in which each county is situated)

*Note that I'm going to assume that the counties (names, FIPS, centroid) have not changed since 2014.*

Then the script loads information about the energy grid for each county provided by the EPA (https://www.epa.gov/egrid/egrid-mapping-files) and identifies which grid sub-region the county centroid lies in.

*We have egrid data for most years back to 2014; if a year is missing data, I'll use data from the previous year*

Then the script loads the NRI Hazard Info (https://hazards.fema.gov/nri/data-resources#hdrDownload) and merges the following predictors into the list of counties:
- Population
- Value of buildings
- Value of agriculture
- Area
- Social Vulnerability Score
- Risks from: Avalange, Costal Flodding, Cold Wave, Hail, Heat Wave, Hurricane, Ice Storm, Landslide, Lightning, Riverine Flooding, Strong Wind, Tornado, Tsunami, Wildfire, and Winter Weather

*Note that there isn't much historical data available - only for 2023, November 2021, August 2021, and October 2020. I'll use the 2023 data for 2023, November 2021 data for 2021 and 2022, and October 2020 data for 2020 and all previous years.*

Then the script loads data from HHS emPOWER (https://empowerprogram.hhs.gov/about-empowermap.html), computes yearly means by county, and merges this data. 

*Note that we have yearly empower data since 2016. I'll use the 2016 data for 2014 and 2015.*

Finally, the script exports the dataframe for each year as a separate CSV files

In [206]:
import geopandas
import pandas as pd

First, we'll load and feature-reduce the list of counties from NWS

In [207]:
#Load the NWS US Counties shapefile
counties = geopandas.read_file('../Data/NWS_US_Counties_Shapefiles')

#Remove the geometry, and time zone variables
counties = counties.drop(columns=['geometry', 'TIME_ZONE'])

#Restrict data to the continental US: Remove all entries in counties where STATE is AS, PR, VI, PW, MH, MP, GU, HI, AK, or FM
counties = counties[~counties['STATE'].isin(['AS', 'PR', 'VI', 'PW', 'MH', 'MP', 'GU', 'FM', 'HI', 'AK'])]

#Convert FIPS in counties to float64 (to facilitate merging with other data sets)
counties['FIPS'] = counties['FIPS'].astype(float)

#Create a new point series in counties using LON as the x coordinate and LAT as the y coordinate
counties['centroid'] = geopandas.points_from_xy(counties['LON'], counties['LAT'])

#Convert counties into a geodataframe and set its crs
counties = geopandas.GeoDataFrame(counties, geometry='centroid')
counties.crs="EPSG:4326"

Next, we'll load the egrid data. The EPA used slightly different file naming conventions for each year, so we'll need to specify the filenames manually

In [208]:
#Load the egrid subregions shapefiles, specify the CRS, and rename columns for consistency
egrid2023 = geopandas.read_file('../Data/egrid2023_subregions')
egrid2023 = egrid2023.to_crs("EPSG:4326")

egrid2022 = geopandas.read_file('../Data/egrid2022_subregions_shapefile')
egrid2022 = egrid2022.to_crs("EPSG:4326")
egrid2022.rename(columns={'ZipSubregi': 'Subregion'}, inplace=True)

egrid2021 = geopandas.read_file('../Data/eGRID2021_subregions_shapefile')
egrid2021 = egrid2021.to_crs("EPSG:4326")
egrid2021.rename(columns={'Shape__Are': 'Shape_Area', 'Shape__Len': 'Shape_Leng', 'SUBRGN': 'Subregion'}, inplace=True)

egrid2020 = geopandas.read_file('../Data/egrid2020_subregions')
egrid2020 = egrid2020.to_crs("EPSG:4326")
egrid2020.rename(columns={'ZipSubregi': 'Subregion'}, inplace=True)
egrid2020.drop(columns=['Shape_Le_1'], inplace=True)

egrid2019 = geopandas.read_file('../Data/egrid2019_subregions')
egrid2019 = egrid2019.to_crs("EPSG:4326")
egrid2019.rename(columns={'ZipSubregi': 'Subregion'}, inplace=True)
egrid2019.drop(columns=['Shape_Le_1'], inplace=True)

egrid2018 = geopandas.read_file('../Data/egrid2018_subregions')
egrid2018 = egrid2018.to_crs("EPSG:4326")
egrid2018.rename(columns={'ZipSubregi': 'Subregion'}, inplace=True)

egrid2016 = geopandas.read_file('../Data/egrid_2016_subregions_shapefiles')
egrid2016 = egrid2016.to_crs("EPSG:4326")
egrid2016.rename(columns={'Subregions': 'Subregion'}, inplace=True)
egrid2016['Shape_Leng'] = ''
egrid2016['Shape_Area'] = ''

egrid2014 = geopandas.read_file('../Data/egrid_subregions')
egrid2014 = egrid2014.to_crs("EPSG:4326")
egrid2014.rename(columns={'zips_for_G': 'Subregion'}, inplace=True)
egrid2014['Shape_Leng'] = ''
egrid2014['Shape_Area'] = ''

#Create a list of egrid files
egrid_files = [egrid2023, egrid2022, egrid2021, egrid2020, egrid2019, egrid2018, egrid2016, egrid2014]

#Drop the variables Shape_Leng and Shape_Area from each of the files in egrid_files
for egrid in egrid_files:
    egrid.drop(columns=['Shape_Leng', 'Shape_Area'], inplace=True)

Next, we'll merge the counties and egrid dataframes

In [209]:
#Use sjoin from geopandas.tools to identify which point value from counties_merged is contained within each multipolygon from the geometry variable in egrid
from geopandas.tools import sjoin

counties2023 = sjoin(counties, egrid2023, how='left')
counties2022 = sjoin(counties, egrid2022, how='left')
counties2021 = sjoin(counties, egrid2021, how='left')
counties2020 = sjoin(counties, egrid2020, how='left')
counties2019 = sjoin(counties, egrid2019, how='left')
counties2018 = sjoin(counties, egrid2018, how='left')
counties2017 = sjoin(counties, egrid2016, how='left')
counties2016 = sjoin(counties, egrid2016, how='left')
counties2015 = sjoin(counties, egrid2014, how='left')
counties2014 = sjoin(counties, egrid2014, how='left')

A manual check revealed several FIPS codes that weren't automatically mapped to an eGRID subregion.

I searched for the county name to identify associated ZIP codes, then entered those into the EPA power profiler (https://www.epa.gov/egrid/power-profiler) to identify the eGRID subregion.

In [210]:
#In all of the counties files, change the missing subregions to the eGRID subregion looked up online
counties_files = [counties2023, counties2022, counties2021, counties2020, counties2019, counties2018, counties2017, counties2016, counties2015, counties2014]

for counties in counties_files:
    counties.loc[counties['FIPS'] == 44005.0, 'Subregion'] = 'NEWE'
    counties.loc[counties['FIPS'] == 45043.0, 'Subregion'] = 'SRVC'
    counties.loc[counties['FIPS'] == 26083.0, 'Subregion'] = 'MROE'
    counties.loc[counties['FIPS'] == 12087.0, 'Subregion'] = 'FRCC'
    counties.loc[counties['FIPS'] == 48007.0, 'Subregion'] = 'ERCT'
    counties.loc[counties['FIPS'] == 53035.0, 'Subregion'] = 'NWPP'
    counties.loc[counties['FIPS'] == 53029.0, 'Subregion'] = 'NWPP'
    counties.loc[counties['FIPS'] == 51735.0, 'Subregion'] = 'SRVC'
    counties.loc[counties['FIPS'] == 37137.0, 'Subregion'] = 'SRVC'
    counties.loc[counties['FIPS'] == 37053.0, 'Subregion'] = 'SRVC'

Next, we'll round the LON and LAT values to the nearest quarter degree to faciliate future merges with ERA5 data

In [211]:
#For each file in counties_files, round LON and LAT to the nearest quarter-degree and re-compute the centroid from these values
for counties in counties_files:
    counties['LON'] = (counties['LON'] * 4).round() / 4
    counties['LAT'] = (counties['LAT'] * 4).round() / 4
    counties['centroid'] = geopandas.points_from_xy(counties['LON'], counties['LAT'])

Next, load the NRI data and merge it with the counties_within data

In [212]:
#Load the data file ../Data/NRI_Table_Counties.csv
nri_2023 = pd.read_csv('../Data/NRI_Table_Counties_2023/NRI_Table_Counties.csv')
nri_2021 = pd.read_csv('../Data/NRI_Table_Counties_2021/NRI_Table_Counties.csv')
nri_2020 = pd.read_csv('../Data/NRI_Table_Counties_2020/NRI_Table_Counties.csv')

#Restrict the nri dataset to only the variables in NRI_Features
NRI_Features = ['STCOFIPS', 'POPULATION', 'BUILDVALUE','AGRIVALUE','AREA','SOVI_SCORE','AVLN_RISKS','CFLD_RISKS','CWAV_RISKS','HAIL_RISKS','HWAV_RISKS','HRCN_RISKS','ISTM_RISKS','LNDS_RISKS','LTNG_RISKS','RFLD_RISKS','SWND_RISKS','TRND_RISKS','TSUN_RISKS','WFIR_RISKS','WNTW_RISKS']
nri_2023 = nri_2023[NRI_Features]
nri_2021 = nri_2021[NRI_Features]
nri_2020 = nri_2020[NRI_Features]

#Merge counties and nri based on the FIPS (in counties) and COUNTYFIPS (in nri) variables
counties2023_nri = counties2023.merge(nri_2023, left_on='FIPS', right_on='STCOFIPS', how='left')
counties2022_nri = counties2022.merge(nri_2021, left_on='FIPS', right_on='STCOFIPS', how='left')
counties2021_nri = counties2021.merge(nri_2021, left_on='FIPS', right_on='STCOFIPS', how='left')
counties2020_nri = counties2020.merge(nri_2020, left_on='FIPS', right_on='STCOFIPS', how='left')
counties2019_nri = counties2019.merge(nri_2020, left_on='FIPS', right_on='STCOFIPS', how='left')
counties2018_nri = counties2018.merge(nri_2020, left_on='FIPS', right_on='STCOFIPS', how='left')
counties2017_nri = counties2017.merge(nri_2020, left_on='FIPS', right_on='STCOFIPS', how='left')
counties2016_nri = counties2016.merge(nri_2020, left_on='FIPS', right_on='STCOFIPS', how='left')
counties2015_nri = counties2015.merge(nri_2020, left_on='FIPS', right_on='STCOFIPS', how='left')
counties2014_nri = counties2014.merge(nri_2020, left_on='FIPS', right_on='STCOFIPS', how='left')

Next, load the empower data, take averages over the course of each year, and do some cleaning

In [213]:
#Load the dataset ../Data/Empower_csvs/2023_HHSemPOWERMapHistoricalDataset.csv
empower2023 = pd.read_csv('../Data/Empower_csvs/2023_HHSemPOWERMapHistoricalDataset.csv')
empower2022 = pd.read_csv('../Data/Empower_csvs/2022_HHSemPOWERMapHistoricalDataset.csv')
empower2021 = pd.read_csv('../Data/Empower_csvs/2021_HHSemPOWERMapHistoricalDataset.csv')
empower2020 = pd.read_csv('../Data/Empower_csvs/2020_HHSemPOWERMapHistoricalDataset.csv')
empower2019 = pd.read_csv('../Data/Empower_csvs/2019_HHSemPOWERMapHistoricalDataset.csv')
empower2018 = pd.read_csv('../Data/Empower_csvs/2018_HHSemPOWERMapHistoricalDataset.csv')
empower2017 = pd.read_csv('../Data/Empower_csvs/2017_HHSemPOWERMapHistoricalDataset.csv')
empower2016 = pd.read_csv('../Data/Empower_csvs/2016_HHSemPOWERMapHistoricalDataset.csv')

empower_files = [empower2023, empower2022, empower2021, empower2020, empower2019, empower2018, empower2017, empower2016]

for df in empower_files:
    #Create a list of features we won't need (including all variables with "Medicare_Benes" in their name). Then drop these columns
    nonfeatures = ['County_FIPS_Code', 'County','State_FIPS_Code', 'State']
    nonfeatures.extend([col for col in df.columns if 'Medicare_Benes' in col])
    df.drop(columns=[col for col in df.columns if col in nonfeatures], inplace=True)

    #For each column in df that is an object, remove commas
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.replace(',', '')

    #For each column in df, convert to an integer
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    
    #Append a new column computed by taking the mean of all columns with "Power_Dependent_Devices_DME" in their name
    df['Power_Dependent_Devices_DME_Mean'] = df[[col for col in df.columns if 'Power_Dependent_Devices_DME' in col]].mean(axis=1)

    #Drop all variables except for FIPS_Code and Power_Dependent_Devices_DME_Mean
    df.drop(columns=[col for col in df.columns if col not in ['FIPS_Code', 'Power_Dependent_Devices_DME_Mean']], inplace=True)






Manually merge the empower data with the counties_nri data

In [214]:
#Merge counties[YEAR]_nri and empower[YEAR] based on the FIPS and FIPS_Code variables
counties2023_nri_empower = counties2023_nri.merge(empower2023, left_on='FIPS', right_on='FIPS_Code', how='left')
counties2022_nri_empower = counties2022_nri.merge(empower2022, left_on='FIPS', right_on='FIPS_Code', how='left')
counties2021_nri_empower = counties2021_nri.merge(empower2021, left_on='FIPS', right_on='FIPS_Code', how='left')
counties2020_nri_empower = counties2020_nri.merge(empower2020, left_on='FIPS', right_on='FIPS_Code', how='left')
counties2019_nri_empower = counties2019_nri.merge(empower2019, left_on='FIPS', right_on='FIPS_Code', how='left')
counties2018_nri_empower = counties2018_nri.merge(empower2018, left_on='FIPS', right_on='FIPS_Code', how='left')
counties2017_nri_empower = counties2017_nri.merge(empower2017, left_on='FIPS', right_on='FIPS_Code', how='left')
counties2016_nri_empower = counties2016_nri.merge(empower2016, left_on='FIPS', right_on='FIPS_Code', how='left')
counties2015_nri_empower = counties2015_nri.merge(empower2016, left_on='FIPS', right_on='FIPS_Code', how='left')
counties2014_nri_empower = counties2014_nri.merge(empower2016, left_on='FIPS', right_on='FIPS_Code', how='left')

Do a bit of cleaning and export

In [215]:
#Make a list of countiesYEAR_nri_empower dataframes
counties_nri_empower_files = [counties2023_nri_empower, counties2022_nri_empower, counties2021_nri_empower, counties2020_nri_empower, counties2019_nri_empower, counties2018_nri_empower, counties2017_nri_empower, counties2016_nri_empower, counties2015_nri_empower, counties2014_nri_empower]

#Drop the variables LON, LAT, index_right, FIPS_Code, and STCOFIPS from counties_merged
for counties in counties_nri_empower_files:
    counties.drop(columns=['LON', 'LAT', 'index_right', 'FIPS_Code', 'STCOFIPS'], inplace=True)

#Export each file as a csv
for i, counties in enumerate(counties_nri_empower_files):
    counties.to_csv(f'../Data/Counties_{2024-i}.csv', index=False)